<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/t5_small_toxic_to_non_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install transformers[torch]

In [2]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/difference_analysis_data_structure")

def format_dataset(example):
    example['input_text'] = example['en_toxic_comment'] + " <sep> " + example['en_neutral_comment']
    example['target_text'] = str(example['edit_ops'])
    return example

formatted_dataset = dataset.map(format_dataset)

In [13]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

def tokenize_data(example):
    tokenized_data = tokenizer(example['input_text'], example['target_text'], truncation=True, padding='max_length', max_length=512)
    tokenized_data['labels'] = tokenized_data['input_ids']
    return tokenized_data

tokenized_dataset = formatted_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/19744 [00:00<?, ? examples/s]

In [11]:
data_collator = lambda data: {
    'input_ids': torch.stack([torch.tensor(item['input_ids']) for item in data]),
    'attention_mask': torch.stack([torch.tensor(item['attention_mask']) for item in data]),
    'labels': torch.stack([torch.tensor(item['labels']) for item in data])
}

In [15]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = T5ForConditionalGeneration.from_pretrained('t5-small')

training_args = Seq2SeqTrainingArguments(
    per_device_train_batch_size=8,
    #per_device_eval_batch_size=8,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    #eval_steps=500,
    save_total_limit=3,
    num_train_epochs=3,
    output_dir="./results",
    #evaluation_strategy="steps",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    data_collator = data_collator
)

trainer.train()

Step,Training Loss
100,1.871200
200,0.036900
300,0.019100
400,0.011300
500,0.007900
600,0.006200
700,0.005300
800,0.005000
900,0.004200
1000,0.003400


TrainOutput(global_step=7404, training_loss=0.02767566895231389, metrics={'train_runtime': 1658.119, 'train_samples_per_second': 35.722, 'train_steps_per_second': 4.465, 'total_flos': 8016565584789504.0, 'train_loss': 0.02767566895231389, 'epoch': 3.0})

In [16]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [18]:
trainer.save_model()

In [19]:
trainer.push_to_hub("t5_small_toxic_to_non_v2")

'https://huggingface.co/HamdanXI/results/tree/main/'